# Generalized Assignment Problem using Lagrangian relaxation

## Business problem   

Solving a generalized assignment problem (GAP), as defined by Wolsey, using this relaxation technic.

The main aim is to show multiple optimization through modifications of different models existing in a single environment, not to show how to solve a GAP problem.

The technic consists in the approximation of a difficult constrained problem by a simpler problem. We will remove difficult constraints by integrating them in the objective function, penalizing it if the constraint is not respected.

The method penalizes violations of inequality constraints using a Lagrange multiplier, which imposes a cost on violations. These added costs are used instead of the strict inequality constraints in the optimization. In practice, this relaxed problem can often be solved more easily than the original problem.

We first solves the standard problem, then show how to reformulate it to meet the Lagrangian Relaxation features.

In [1]:
import docplex.mp

In [2]:
# Data Model
# 5 x 3
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]
# 1 x 3
B = [15, 15, 15]
# 5 x 3
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]

In [3]:
#We will firt create an optimization problem, composed of 2 basic constraints blocks, then we will resolve it using Lagrangian Relaxation on 1 of the constraints block.
from docplex.mp.model import Model

#Create the DOcplex model
mdl = Model("AMC GAP")

In [4]:
print("#As={}, #Bs={}, #Cs={}".format(len(A), len(B), len(C)))
number_of_cs = len(C)
# Define the decision variables
x_vars = [mdl.binary_var_list(c, name=None) for c in C]

#As=5, #Bs=3, #Cs=5


In [5]:
# Define the business constraints
cts = mdl.add_constraints(mdl.sum(i) <= 1 for i in x_vars)

mdl.add_constraints(mdl.sum(x_vars[i][j] * A[i][j] for i in range(number_of_cs)) <= bs for j, bs in enumerate(B))

# objective
total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(C, x_vars))
mdl.maximize(total_profit)
mdl.print_information()

Model: AMC GAP
 - number of variables: 15
   - binary=15, integer=0, continuous=0
 - number of constraints: 8
   - linear=8
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [6]:
# Solve the model 
s = mdl.solve()
assert s is not None
obj = s.objective_value
print("GAP with no relaxation run OK, best objective is: {:g}".format(obj))

GAP with no relaxation run OK, best objective is: 46


In [7]:
print(s)

solution for: AMC GAP
objective: 46
x2=1
x5=1
x7=1
x12=1



## Solve the model with Lagrangian Relaxation method

For the demonstration of the Lagrangian Relaxation let us suppose that this model was hard to solve for CPLEX.
So we will approximate this problem by doing an iterative model - the objective is modified at each iteration. 

In [8]:
# We first remove the problematic constraints from the model
for ct in cts:
    mdl.remove_constraint(ct)

In [9]:
#p_vars are the penalties attached to violating the constraints
p_vars = mdl.continuous_var_list(C, name='penalties')  # new for relaxation

In [10]:
# new version of the approximated constraint where we apply the penalties
mdl.add_constraints(mdl.sum(xv) == 1 - pv for xv, pv in zip(x_vars, p_vars))

''

In [11]:
#Define the maximum number of iterations
max_iters = 10

In [12]:
number_of_cs = len(C)
c_range = range(number_of_cs)

In [13]:
# Langrangian relaxation loop 
eps = 1e-6
loop_count = 0
best = 0
initial_multiplier = 1
multipliers = [initial_multiplier] * len(C)

# Objective function
# We will use the key perfromance indicator (kpi) as
# total_profit = mdl.sum(mdl.sum(x_vars[task][worker] * C[task][worker]) for task, worker in zip(tasks, workers))
total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(C, x_vars))
mdl.add_kpi(total_profit, "Total profit")
print("starting the loop")

starting the loop


In [14]:
while loop_count <= max_iters:
    loop_count += 1
    # Rebuilt at each loop iteration
    total_penalty = mdl.scal_prod(p_vars, multipliers)
    
    mdl.maximize(total_profit + total_penalty)
    s = mdl.solve()
    if not s:
        print("*** solve fails, stopping at iteration: %d" % loop_count)
        break
    best = s.objective_value
    penalties = [pv.solution_value for pv in p_vars]
    print('%d> new lagrangian iteration:\n\t obj=%g, m=%s, p=%s' % (loop_count, best, str(multipliers), str(penalties)))

    do_stop = True
    justifier = 0
    for k in c_range:
        penalized_violation = penalties[k] * multipliers[k]
        if penalized_violation >= eps:
            do_stop = False
            justifier = penalized_violation
            break

    if do_stop:
        print("* Lagrangian relaxation succeeds, best={:g}, penalty={:g}, #iterations={}"
                .format(best, total_penalty.solution_value, loop_count))
        break
    else:
        # Update multipliers and start the loop again.
        scale_factor = 1.0 / float(loop_count)
        multipliers = [max(multipliers[i] - scale_factor * penalties[i], 0.) for i in c_range]
        print('{0}> -- loop continues, m={1!s}, justifier={2:g}'.format(loop_count, multipliers, justifier))

1> new lagrangian iteration:
	 obj=47, m=[1, 1, 1, 1, 1], p=[0, 0, 0, 0, 1.0]
1> -- loop continues, m=[1.0, 1.0, 1.0, 1.0, 0.0], justifier=1
2> new lagrangian iteration:
	 obj=46, m=[1.0, 1.0, 1.0, 1.0, 0.0], p=[0, 0, 0, 0, 1.0]
* Lagrangian relaxation succeeds, best=46, penalty=0, #iterations=2


In [15]:
print(best)

46.0
